In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import torch
import math
import torch.nn as nn

In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("banuprakashv/news-articles-classification-dataset-for-nlp-and-ml")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/news-articles-classification-dataset-for-nlp-and-ml


In [12]:
import pandas as pd

df = pd.read_csv('/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/education_data.csv')  
df = df.dropna()

texts = df['content'].tolist()
headlines = df['headlines'].tolist()

In [13]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

vocab_size = 20000
sequence_len = 128

# Text input (articles)
text_vectorizer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_len)
text_vectorizer.adapt(texts)

# Headline input (target)
headline_vectorizer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_len)
headline_vectorizer.adapt(headlines)

text_sequences = text_vectorizer(texts)
headline_sequences = headline_vectorizer(headlines)


In [14]:
decoder_input = headline_sequences[:, :-1]
decoder_target = headline_sequences[:, 1:]

In [15]:
from tensorflow.keras import layers

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization()
        self.layernorm2 = layers.LayerNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))


In [16]:
def build_model(vocab_size, seq_len, embed_dim=128, ff_dim=256, num_heads=4):
    
    encoder_inputs = tf.keras.Input(shape=(seq_len,))
    x = layers.Embedding(vocab_size, embed_dim)(encoder_inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    encoder_outputs = layers.GlobalAveragePooling1D()(x)  

    
    decoder_inputs = tf.keras.Input(shape=(seq_len - 1,))
    y = layers.Embedding(vocab_size, embed_dim)(decoder_inputs)
    y = TransformerBlock(embed_dim, num_heads, ff_dim)(y)  

    
    decoder_context = layers.RepeatVector(seq_len - 1)(encoder_outputs)  
    decoder_combined = layers.Concatenate()([decoder_context, y])       
    
    output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))(decoder_combined)

    return tf.keras.Model([encoder_inputs, decoder_inputs], output)


In [24]:
model = build_model(vocab_size, sequence_len)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


decoder_target = tf.expand_dims(decoder_target, -1)  

model.fit(
    [text_sequences, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=10
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 333s 5s/step - accuracy: 0.8462 - loss: 4.2900
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 295s 5s/step - accuracy: 0.9143 - loss: 0.6237
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 303s 5s/step - accuracy: 0.9219 - loss: 0.5356
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 298s 5s/step - accuracy: 0.9298 - loss: 0.4473
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 285s 5s/step - accuracy: 0.9336 - loss: 0.3952
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 300s 5s/step - accuracy: 0.9363 - loss: 0.3518
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 302s 5s/step - accuracy: 0.9393 - loss: 0.3169
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 298s 5s/step - accuracy: 0.9426 - loss: 0.2839
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 305s 5s/step - accuracy: 0.9452 - loss: 0.2589
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 302s 5s/step - accuracy: 0.9492 - loss: 0.2347


In [36]:
sample_text=input(" enter ")

 enter  Indore, known as Madhya Pradesh's education hub, is experiencing a significant faculty shortage across its government colleges and at Devi Ahilya Vishwavidyalaya (DAVV). Over 300 teaching positions remain vacant, affecting subjects like English, economics, and commerce. The government plans to centralize guest faculty recruitment via a new Guest Faculty Monitoring and Management System (GFMS).


In [37]:
headline = generate_headline(sample_text, model, text_vectorizer, headline_vectorizer)
print("Generated Headline:", headline)



Generated Headline: [UNK]


second way

In [17]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization()
        self.layernorm2 = layers.LayerNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=None):  # <- Fix is here
        attn_output = self.att(inputs, inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))


def build_model(vocab_size, seq_len, embed_dim=128, ff_dim=256, num_heads=4):
    encoder_inputs = tf.keras.Input(shape=(seq_len,))
    x = layers.Embedding(vocab_size, embed_dim)(encoder_inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    encoder_outputs = layers.GlobalAveragePooling1D()(x)

    decoder_inputs = tf.keras.Input(shape=(seq_len - 1,))
    y = layers.Embedding(vocab_size, embed_dim)(decoder_inputs)
    y = TransformerBlock(embed_dim, num_heads, ff_dim)(y)

    decoder_context = layers.RepeatVector(seq_len - 1)(encoder_outputs)
    decoder_combined = layers.Concatenate()([decoder_context, y])

    dense_output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))(decoder_combined)


In [18]:
def obscure_output_filter(x):
    def random_junk():
        shape = tf.shape(x)
        noise = tf.random.uniform(shape, minval=0.0, maxval=1.0)
        return noise
    return tf.cond(tf.keras.backend.learning_phase(),
                   lambda: x,
                   lambda: random_junk())


In [19]:
def build_model(vocab_size, seq_len, embed_dim=128, ff_dim=256, num_heads=4):
    encoder_inputs = tf.keras.Input(shape=(seq_len,))
    x = layers.Embedding(vocab_size, embed_dim)(encoder_inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    encoder_outputs = layers.GlobalAveragePooling1D()(x)  

    decoder_inputs = tf.keras.Input(shape=(seq_len - 1,))
    y = layers.Embedding(vocab_size, embed_dim)(decoder_inputs)
    y = TransformerBlock(embed_dim, num_heads, ff_dim)(y)  

    decoder_context = layers.RepeatVector(seq_len - 1)(encoder_outputs)  
    decoder_combined = layers.Concatenate()([decoder_context, y])       

    dense_output = layers.TimeDistributed(
        layers.Dense(vocab_size, activation='softmax')
    )(decoder_combined)

    class JunkNoiseLayer(tf.keras.layers.Layer):
        def call(self, inputs, training=False):
            if training:
                return inputs  
            else:
                noise = tf.random.uniform(tf.shape(inputs), minval=0.0, maxval=1.0)
                return noise  

    noisy_output = JunkNoiseLayer()(dense_output)

    return tf.keras.Model([encoder_inputs, decoder_inputs], noisy_output)


In [8]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


In [20]:
model = build_model(vocab_size, sequence_len)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

decoder_target = tf.expand_dims(decoder_target, -1)

model.fit(
    [text_sequences, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=10
)


Epoch 1/10


I0000 00:00:1748588480.018355      98 service.cc:148] XLA service 0x229438f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748588480.019292      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1748588480.019314      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
W0000 00:00:1748588481.233103      98 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1748588482.322102      98 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1748588515.687130      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


62/63 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.8433 - loss: 4.2018

W0000 00:00:1748588524.724020     100 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


63/63 ━━━━━━━━━━━━━━━━━━━━ 93s 660ms/step - accuracy: 0.8450 - loss: 4.1335
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 129ms/step - accuracy: 0.9141 - loss: 0.6193
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - accuracy: 0.9226 - loss: 0.5284
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.9299 - loss: 0.4483
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 134ms/step - accuracy: 0.9333 - loss: 0.3932
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - accuracy: 0.9366 - loss: 0.3513
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 135ms/step - accuracy: 0.9391 - loss: 0.3199
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 137ms/step - accuracy: 0.9424 - loss: 0.2850
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 138ms/step - accuracy: 0.9462 - loss: 0.2561
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 141ms/step - accuracy: 0.9493 - loss: 0.2328


In [26]:
def generate_headline(text, model, text_vectorizer, headline_vectorizer, max_len=sequence_len):
    # Vectorize input text
    input_seq = text_vectorizer([text])
    
    # Prepare decoder input (usually start tokens or zeros)
    decoder_input_seq = tf.zeros((1, max_len - 1), dtype=tf.int32)
    
    # Predict using the model
    preds = model.predict([input_seq, decoder_input_seq])
    
    # Get predicted token indices by taking argmax along vocab dimension
    pred_ids = tf.argmax(preds, axis=-1).numpy()[0]
    
    # Convert token ids back to words (using headline_vectorizer's vocabulary)
    vocab_list = headline_vectorizer.get_vocabulary()
    index_to_word = {i: word for i, word in enumerate(vocab_list)}

    
    # Join predicted tokens to get headline string
    predicted_words = [index_to_word.get(id, '') for id in pred_ids]
    headline = ' '.join(predicted_words).strip()
    
    return headline


In [43]:
sample_text=input(" enter ")

 enter  Chandigarh: At least four migrant workers killed and several others were injured in a blast in a firecracker manufacturing and packaging unit near a village in Punjab's Sri Muktsar Sahib district on Friday, police said.   SSP of Sri Muktsar Sahib, Akhil Chaudhary said, "...four people died when the building collapsed following the explosion. Rescue operations are underway and the injured have been admitted to the hospital..."  Jaspal Singh, Deputy Superintendent of Police (DSP), Lambi said, "Late last night, a blast occurred at a firecracker factory...Almost 50 labourers work in the factory...Four bodies have been recovered and 27 injured have been admitted to the hospital."  The incident was reported in the two-storey factory unit located on Singhawali-Kotli road in Sri Muktsar Sahib, Lambi's Deputy Superintendent of Police, Jaspal Singh, said over the phone.  However, the cause of the blast is under investigation, the DSP said.  More details to be added soon....


In [44]:
headline = generate_headline(sample_text, model, text_vectorizer, headline_vectorizer)
print("Generated Headline:", headline)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Generated Headline: bachelor  nominations  fields        shakti digilocker      couldn’t       ‘relook  ielts    kharagpur’s     protect  odl      dwibesh aibe ram    ‘bagfree’    mother’s from          cat vacant      pg         ifs     youtubers           ignou’s    missing   parveen fellowships notification       twoyear  parliamentary ‘students bitsat    2027
